In [7]:
import nltk
import re
from pattern.en import lemma, lexeme
from nltk.stem import WordNetLemmatizer

nltk.download('all')

# Reading Text File
w = []
with open('final.txt', 'r', encoding="utf8") as f:
    file_name_data = f.read()
    file_name_data = file_name_data.lower()
    w = re.findall('\w+', file_name_data)

# Counting Word Frequency
def counting_words(words):
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

# Calculating Word Probabilities
def prob_cal(word_count_dict):
    probs = {}
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    return probs

# LemmWord: extracting and adding root word i.e. Lemma using pattern module
def LemmWord(word):
    return list(lexeme(wd) for wd in word.split())[0]

# Deleting letters from the words
def DeleteLetter(word):
    delete_list = []
    split_list = []

    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))

    for a, b in split_list:
        delete_list.append(a + b[1:])
    return delete_list

# Switching two letters in a word
def Switch_(word):
    split_list = []
    switch_l = []

    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))

    switch_l = [a + b[1] + b[0] + b[2:] for a, b in split_list if len(b) >= 2]
    return switch_l

# Replacing the letter one-by-one from the list of alphs
def Replace_(word):
    split_l = []
    replace_list = []

    for i in range(len(word)):
        split_l.append((word[0:i], word[i:]))
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    replace_list = [a + l + (b[1:] if len(b) > 1 else '') for a, b in split_l if b for l in alphs]
    return replace_list

# Making pairs of the split words and storing new words in a list with one new character at each location
def insert_(word):
    split_l = []
    insert_list = []

    for i in range(len(word) + 1):
        split_l.append((word[0:i], word[i:]))

    alphs = 'abcdefghijklmnopqrstuvwxyz'
    insert_list = [a + l + b for a, b in split_l for l in alphs]
    return insert_list

# Collecting all the words in a set(so that no word will repeat)
def colab_1(word, allow_switches=True):
    colab_1 = set()
    colab_1.update(DeleteLetter(word))
    if allow_switches:
        colab_1.update(Switch_(word))
    colab_1.update(Replace_(word))
    colab_1.update(insert_(word))
    return colab_1

# Collecting words using by allowing switches
def colab_2(word, allow_switches=True):
    colab_2 = set()
    edit_one = colab_1(word, allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = colab_1(w, allow_switches=allow_switches)
            colab_2.update(edit_two)
    return colab_2

# Only storing those values which are in the vocab
def get_corrections(word, probs, vocab, n=5, threshold_prob=0.001, threshold_similarity=0.1):
    suggested_word = []
    best_suggestion = []
    suggested_word = list(
        (word in vocab and word) or colab_1(word).intersection(vocab)
        or colab_2(word).intersection(vocab))

    # Filtering suggestions based on probability and similarity
    best_suggestion = [[s, probs[s]] for s in list(reversed(suggested_word)) if probs[s] > threshold_prob]
    best_suggestion = [[s, prob] for s, prob in best_suggestion if prob >= max(best_suggestion, key=lambda x: x[1])[1] - threshold_similarity]

    # Sorting by probability
    best_suggestion = sorted(best_suggestion, key=lambda x: x[1], reverse=True)

    # Returning only top N suggestions
    return best_suggestion[:n]

# Input
my_word = input("Enter any word:")

# Vocabulary
main_set = set(w)

# Counting word function
word_count = counting_words(main_set)

# Calculating probability
probs = prob_cal(word_count)

# Only storing correct words
tmp_corrections = get_corrections(my_word, probs, main_set, n=5)

# Print top suggestions made by the Autocorrect
for suggestion, probability in tmp_corrections:
    print(f"Suggested Word: {suggestion}, Probability: {probability:.6f}")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Varun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Varun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Varun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Varun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Varun\AppData\Roaming\nltk_data...
[nltk_data]    | 

Enter any word:guess


In [6]:
from nltk.tokenize import word_tokenize
from pattern.en import suggest

def correct_spelling(word):
    tokens = word_tokenize(word)
    
    corrected_words = []
    for token in tokens:
        suggestions = suggest(token)
        if suggestions:
            corrected_words.append(suggestions[0][0])
        else:
            corrected_words.append(token)
    
    return corrected_words

word = input("Enter any word: ")
corrected_words = correct_spelling(word)
print("Corrected Words:", corrected_words)


Enter any word: hella


RuntimeError: generator raised StopIteration